## Importing Libraries

In [1]:
import numpy as np
import tensorflow as tf
from sklearn import preprocessing

raw_data = np.loadtxt('Audiobooks-data.csv',delimiter=',')

unscaled_inputs_all = raw_data[:,1:-1]
targets_all = raw_data[:,-1]

## Preprocessing the Data

In [2]:
# Balancing the data
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)

unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all,indices_to_remove,axis=0)
targets_equal_priors = np.delete(targets_all,indices_to_remove,axis=0)

# Standardize the inputs
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

# Shuffle the data
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)
shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

# Split the dataset into Train, Validation and Test
# use 80-10-10 split 
samples_count = shuffled_inputs.shape[0]
train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
target_samples_count = samples_count - train_samples_count - validation_samples_count
# Training inputs and targets
train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]
# Validation inputs and targets
validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]
# Test inputs and targets
test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

# Save Training, Validation and Test dataset in *.npz file
np.savez('Audiobooks_data_train',inputs=train_inputs,targets=train_targets)
np.savez('Audiobooks_data_validation',inputs=validation_inputs,targets=validation_targets)
np.savez('Audiobooks_data_test',inputs=test_inputs,targets=test_targets)

## Data

In [3]:
# Load the training dataset
npz = np.load('Audiobooks_data_train.npz')
train_inputs, train_targets = npz['inputs'].astype(np.float),npz['targets'].astype(np.int)

# Load the validation dataset
npz = np.load('Audiobooks_data_validation.npz')
validation_inputs, validation_targets = npz['inputs'].astype(np.float),npz['targets'].astype(np.int)

# Load the test dataset
npz = np.load('Audiobooks_data_test.npz')
test_inputs, test_targets = npz['inputs'].astype(np.float),npz['targets'].astype(np.int)

## Model

In [12]:
input_size = 10
output_size = 2
hiddel_layer_size = 150

model = tf.keras.Sequential([
                            tf.keras.layers.Dense(hiddel_layer_size,activation='relu'),
                            tf.keras.layers.Dense(hiddel_layer_size,activation='relu'),
                            tf.keras.layers.Dense(hiddel_layer_size,activation='sigmoid'),
                            tf.keras.layers.Dense(hiddel_layer_size,activation='sigmoid'),
                            tf.keras.layers.Dense(output_size,activation='softmax')
                            ])

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

batch_size = 100
max_epochs = 100
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs,
          train_targets,
         batch_size=batch_size,
         epochs = max_epochs,
         callbacks=[early_stopping],
         validation_data = (validation_inputs,validation_targets),
         verbose=2) 

Train on 3579 samples, validate on 447 samples
Epoch 1/100
3579/3579 - 1s - loss: 0.6647 - accuracy: 0.5851 - val_loss: 0.5094 - val_accuracy: 0.7763
Epoch 2/100
3579/3579 - 0s - loss: 0.4282 - accuracy: 0.7731 - val_loss: 0.3659 - val_accuracy: 0.7852
Epoch 3/100
3579/3579 - 0s - loss: 0.3771 - accuracy: 0.7857 - val_loss: 0.3525 - val_accuracy: 0.8054
Epoch 4/100
3579/3579 - 0s - loss: 0.3529 - accuracy: 0.8092 - val_loss: 0.3516 - val_accuracy: 0.8031
Epoch 5/100
3579/3579 - 0s - loss: 0.3428 - accuracy: 0.8120 - val_loss: 0.3296 - val_accuracy: 0.8210
Epoch 6/100
3579/3579 - 0s - loss: 0.3452 - accuracy: 0.7991 - val_loss: 0.3255 - val_accuracy: 0.8322
Epoch 7/100
3579/3579 - 0s - loss: 0.3379 - accuracy: 0.8111 - val_loss: 0.3545 - val_accuracy: 0.7875
Epoch 8/100
3579/3579 - 0s - loss: 0.3365 - accuracy: 0.8128 - val_loss: 0.3226 - val_accuracy: 0.8143
Epoch 9/100
3579/3579 - 0s - loss: 0.3321 - accuracy: 0.8100 - val_loss: 0.3383 - val_accuracy: 0.8076
Epoch 10/100
3579/3579 - 0

## Test the model

In [21]:
test_loss, test_accuracy = model.evaluate(test_inputs,test_targets,verbose=0)
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.32. Test accuracy: 82.37%
